In [40]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.optimize import curve_fit
import seaborn as sns
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [114]:
df=pd.read_csv('ams_LIMPIO.csv')
df.head(5)

,Unnamed: 0,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,...,room_type,bathrooms_text,amenities,calendar_updated,has_availability,calendar_last_scraped,first_review,last_review,license,instant_bookable
0,0,306852,20240610194833,1141485,3.0,3.0,NaN,52.35931,4.89844,4.0,...,Entire home/apt,1.5 baths,"[""Hair dryer"", ""Iron"", ""Microwave"", ""Smoke ala...",-,t,2024-06-11,2012-09-14,2023-04-01,0363 33C3 B936 3911 FC42,f
1,1,307497,20240610194833,1582128,2.0,2.0,NaN,52.39185,4.88690,2.9,...,Entire home/apt,2.5 baths,"[""Patio or balcony"", ""Pets allowed"", ""Stove"", ...",-,t,2024-06-11,2012-09-10,2024-05-12,Exempt,f
2,2,307621,20240610194833,1141485,3.0,3.0,NaN,52.35871,4.89616,2.0,...,Entire home/apt,1 bath,"[""Hair dryer"", ""Iron"", ""High chair"", ""Smoke al...",-,t,2024-06-11,2013-04-02,2023-03-27,0363 33C3 B936 3911 FC42,f
3,3,308028,20240610194833,1141485,3.0,3.0,NaN,52.35972,4.89750,2.0,...,Private room,Private half-bath,"[""Hair dryer"", ""Iron"", ""Microwave"", ""Pets allo...",-,t,2024-06-11,2014-03-25,2023-02-28,0363 33C3 B936 3911 FC42,f
4,4,311124,20240610194833,1600010,1.0,1.0,NaN,52.37104,4.87713,2.0,...,Entire home/apt,1 bath,"[""Iron"", ""MI box sound system"", ""Wine glasses""...",-,t,2024-06-11,2012-05-01,2024-05-27,0363 59D8 7D30 6CFA DC81,f


In [42]:
df['host_is_superhost'] = df['host_is_superhost'].replace({
    't': 'yes',
    'f': 'no',
    'VALIDACIÓN PENDIENTE': np.nan
})

df['host_has_profile_pic'] = df['host_has_profile_pic'].replace({
    't': 'yes',
    'f': 'no',
    'INFORMACIÓN FALTANTE': np.nan
})

df['host_identity_verified'] = df['host_identity_verified'].replace({
    't': 'yes',
    'f': 'no',
    'INFORMACIÓN FALTANTE': np.nan
})

df['has_availability'] = df['has_availability'].replace({
    't': 'yes',
    'f': 'no',
    'INFORMACIÓN NO DISPONIBLE': np.nan
})

df['instant_bookable'] = df['instant_bookable'].replace({
    't': 'yes',
    'f': 'no',
    'INFORMACIÓN NO DISPONIBLE': np.nan
})

df =df.fillna(method="bfill")
df =df.fillna(method="ffill")

<ipython-input-42-d98a12bdaf1e>:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="bfill")
<ipython-input-42-d98a12bdaf1e>:32: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="ffill")


In [4]:
df['host_has_profile_pic'].unique()

array(['yes', 'no'], dtype=object)

In [5]:
df["host_has_profile_pic"].head(30)

,host_has_profile_pic
0,yes
1,yes
2,yes
3,yes
4,yes
5,yes
6,yes
7,yes
8,yes
9,yes


MODELO 1 host_is_superhost

In [6]:
#Declarar las variables independientes y la dependiente
Vars_Indep=df[["price", "reviews_per_month", "bedrooms"]]
Var_Dep=df["host_is_superhost"]
X=Vars_Indep
y=Var_Dep

#Dividimos el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=None)

#Se escalan los datos
escalar=StandardScaler()
X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

#Entrenar el modelo
algoritmo.fit(X_train, y_train)

y_pred=algoritmo.predict(X_test)
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print("Matriz de confusión")
print(matriz)

Matriz de confusión
[[2245   57]
 [ 365  126]]


In [7]:
#calcular precision del modelo
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average="binary", pos_label="yes")
print("precision del modelo:")
print(precision)

precision del modelo:
0.6885245901639344


In [8]:
#calcular exactitud
from sklearn.metrics import accuracy_score
exactitud=accuracy_score(y_test, y_pred)
print("exactitud del modelo")
print(exactitud)

exactitud del modelo
0.8489079842463301


In [9]:
#calcular la sensibilidad
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="yes")
print("sensibilidad del modelo")
print(sensibilidad)

sensibilidad del modelo
0.25661914460285135


MODELO 2 host_has_profile_pic

In [10]:
#Declarar las variables independientes y la dependiente
Vars_Indep=df[["price", "reviews_per_month", "bedrooms"]]
Var_Dep=df["host_has_profile_pic"]
X=Vars_Indep
y=Var_Dep

#Dividimos el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=None)

#Se escalan los datos
escalar=StandardScaler()
X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

#Entrenar el modelo
algoritmo.fit(X_train, y_train)

y_pred=algoritmo.predict(X_test)
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print("Matriz de confusión")
print(matriz)

Matriz de confusión
[[   0   33]
 [   0 2760]]


In [11]:
#calcular precision del modelo
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average="binary", pos_label="yes")
print("precision del modelo:")
print(precision)

precision del modelo:
0.9881847475832438


In [12]:
#calcular exactitud
from sklearn.metrics import accuracy_score
exactitud=accuracy_score(y_test, y_pred)
print("exactitud del modelo")
print(exactitud)

exactitud del modelo
0.9881847475832438


In [13]:
#calcular la sensibilidad
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="yes")
print("sensibilidad del modelo")
print(sensibilidad)

sensibilidad del modelo
1.0


MODELO 3 host_identity_verified

In [14]:
#Declarar las variables independientes y la dependiente
Vars_Indep=df[["price", "reviews_per_month", "bedrooms"]]
Var_Dep=df["host_identity_verified"]
X=Vars_Indep
y=Var_Dep

#Dividimos el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=None)

#Se escalan los datos
escalar=StandardScaler()
X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

#Entrenar el modelo
algoritmo.fit(X_train, y_train)

y_pred=algoritmo.predict(X_test)
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print("Matriz de confusión")
print(matriz)

Matriz de confusión
[[   0   87]
 [   0 2706]]


In [15]:
#calcular precision del modelo
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average="binary", pos_label="yes")
print("precision del modelo:")
print(precision)

precision del modelo:
0.9688506981740065


In [16]:
#calcular exactitud
from sklearn.metrics import accuracy_score
exactitud=accuracy_score(y_test, y_pred)
print("exactitud del modelo")
print(exactitud)

exactitud del modelo
0.9688506981740065


In [17]:
#calcular la sensibilidad
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="yes")
print("sensibilidad del modelo")
print(sensibilidad)

sensibilidad del modelo
1.0


MODELO has_availability

In [18]:
#Declarar las variables independientes y la dependiente
Vars_Indep=df[["price", "reviews_per_month", "bedrooms"]]
Var_Dep=df["has_availability"]
X=Vars_Indep
y=Var_Dep

#Dividimos el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=None)

#Se escalan los datos
escalar=StandardScaler()
X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

#Entrenar el modelo
algoritmo.fit(X_train, y_train)

y_pred=algoritmo.predict(X_test)
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print("Matriz de confusión")
print(matriz)

Matriz de confusión
[[   0   14]
 [   0 2779]]


In [19]:
#calcular precision del modelo
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average="binary", pos_label="yes")
print("precision del modelo:")
print(precision)

precision del modelo:
0.9949874686716792


In [20]:
#calcular exactitud
from sklearn.metrics import accuracy_score
exactitud=accuracy_score(y_test, y_pred)
print("exactitud del modelo")
print(exactitud)

exactitud del modelo
0.9949874686716792


In [21]:
#calcular la sensibilidad
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="yes")
print("sensibilidad del modelo")
print(sensibilidad)

sensibilidad del modelo
1.0


------------------------------

---------------

---------------

Creación de categorías a partir de clases

host_acceptance_rate

In [115]:
# Asegurarse de que la columna sea de tipo string
df['host_acceptance_rate'] = df['host_acceptance_rate'].astype(str)

# Eliminar el símbolo de porcentaje y convertir a numérico
df['host_acceptance_rate'] = pd.to_numeric(df['host_acceptance_rate'].str.replace('%', ''), errors='coerce')

df['host_acceptance_rate'] = df['host_acceptance_rate'] / 100
df["host_acceptance_rate"] = df["host_acceptance_rate"].fillna(round(df["host_acceptance_rate"].mean(), 1))

In [116]:
Max=df["host_acceptance_rate"].max()
Min=df["host_acceptance_rate"].min()
Limites=[Min, Max]
Limites

[0.0, 1.0]

In [117]:
#categorizar las variables
#declarar dos intervalos

intervalos=np.linspace(-0.1, 1.1,3)
intervalos

#Crear las categorías
categorias=["Aceptacion baja", "Aceptacion alta"]

df["host_acceptance_rate"]=pd.cut(x=df["host_acceptance_rate"], bins=intervalos, labels=categorias)

df["host_acceptance_rate"]

,host_acceptance_rate
0,Aceptacion alta
1,Aceptacion alta
2,Aceptacion alta
3,Aceptacion alta
4,Aceptacion alta
...,...
9305,Aceptacion alta
9306,Aceptacion alta
9307,Aceptacion alta
9308,Aceptacion alta


host_response_rate

In [118]:
# Asegurarse de que la columna sea de tipo string
df['host_response_rate'] = df['host_response_rate'].astype(str)

# Eliminar el símbolo de porcentaje y convertir a numérico
df['host_response_rate'] = pd.to_numeric(df['host_response_rate'].str.replace('%', ''), errors='coerce')

df['host_response_rate'] = df['host_response_rate'] / 100
df["host_response_rate"] = df["host_response_rate"].fillna(round(df["host_response_rate"].mean(), 1))

In [119]:
Max=df["host_response_rate"].max()
Min=df["host_response_rate"].min()
Limites=[Min, Max]
Limites

[0.0, 1.0]

In [120]:
#categorizar las variables
#declarar dos intervalos

intervalos=np.linspace(-0.1, 1.1,3)
intervalos

#Crear las categorías
categorias=["Respuesta baja", "Respuesta alta"]

df["host_response_rate"]=pd.cut(x=df["host_response_rate"], bins=intervalos, labels=categorias)

df["host_response_rate"]

,host_response_rate
0,Respuesta alta
1,Respuesta baja
2,Respuesta alta
3,Respuesta alta
4,Respuesta alta
...,...
9305,Respuesta alta
9306,Respuesta alta
9307,Respuesta alta
9308,Respuesta alta


host_total_listings_count

In [121]:
Max=df["host_total_listings_count"].max()
Min=df["host_total_listings_count"].min()
Limites=[Min, Max]
Limites

[1.0, 127.0]

In [ ]:
#categorizar las variables
#declarar dos intervalos

intervalos=np.linspace(0.9, 127.1,3)
intervalos

#Crear las categorías
categorias=["Aceptacion baja", "Aceptacion alta"]

df["host_total_listings_count"]=pd.cut(x=df["host_total_listings_count"], bins=intervalos, labels=categorias)

df["host_total_listings_count"]

number_of_reviews

In [123]:
Max=df["number_of_reviews"].max()
Min=df["number_of_reviews"].min()
Limites=[Min, Max]
Limites

[0.0, 395.0]

In [124]:

#categorizar las variables
#declarar dos intervalos

intervalos=np.linspace(-0.1, 395.1,3)
intervalos

#Crear las categorías
categorias=["Reseñas bajas", "Reseñas altas"]

df["number_of_reviews"]=pd.cut(x=df["number_of_reviews"], bins=intervalos, labels=categorias)

df["number_of_reviews"]

,number_of_reviews
0,Reseñas bajas
1,Reseñas bajas
2,Reseñas bajas
3,Reseñas bajas
4,Reseñas bajas
...,...
9305,Reseñas bajas
9306,Reseñas bajas
9307,Reseñas bajas
9308,Reseñas bajas


latitude

In [125]:
Max=df["latitude"].max()
Min=df["latitude"].min()
Limites=[Min, Max]
Limites

[52.31458, 52.41878]

In [126]:
#categorizar las variables
#declarar dos intervalos

intervalos=np.linspace(52.30, 52.5,3)
intervalos

#Crear las categorías
categorias=["Latitud bajas", "Latitud altas"]

df["latitude"]=pd.cut(x=df["latitude"], bins=intervalos, labels=categorias)

df["latitude"]

,latitude
0,Latitud bajas
1,Latitud bajas
2,Latitud bajas
3,Latitud bajas
4,Latitud bajas
...,...
9305,Latitud bajas
9306,Latitud bajas
9307,Latitud altas
9308,Latitud bajas


In [127]:
df.isnull().sum()

,0
Unnamed: 0,0
id,0
scrape_id,0
host_id,0
host_listings_count,0
...,...
calendar_last_scraped,0
first_review,0
last_review,0
license,0


In [108]:
df =df.fillna(method="bfill")
df =df.fillna(method="ffill")
df.isnull().sum()

<ipython-input-108-f51a50de52de>:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="bfill")
<ipython-input-108-f51a50de52de>:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df =df.fillna(method="ffill")


,0
Unnamed: 0,0
id,0
scrape_id,0
host_id,0
host_listings_count,0
...,...
calendar_last_scraped,0
first_review,0
last_review,0
license,0


---------------

---------------

---------------

Regresión logistica

host_acceptance_rate

In [128]:
#Declarar las variables independientes y la dependiente
Vars_Indep=df[["price", "accommodates", "bedrooms"]]
Var_Dep=df["host_acceptance_rate"]
X=Vars_Indep
y=Var_Dep

#Dividimos el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=None)

#Se escalan los datos
escalar=StandardScaler()
X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

#Entrenar el modelo
algoritmo.fit(X_train, y_train)

y_pred=algoritmo.predict(X_test)
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print("Matriz de confusión")
print(matriz)

Matriz de confusión
[[2222    0]
 [ 571    0]]


In [131]:
#calcular precision del modelo
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average="binary", pos_label="Aceptacion alta")
print("precision del modelo:")
print(precision)

precision del modelo:
0.7955603293949158


In [132]:
#calcular exactitud
from sklearn.metrics import accuracy_score
exactitud=accuracy_score(y_test, y_pred)
print("exactitud del modelo")
print(exactitud)

exactitud del modelo
0.7955603293949158


In [133]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="Aceptacion alta")
print("sensibilidad del modelo")
print(sensibilidad)

sensibilidad del modelo
1.0


host_response_rate

In [134]:
#Declarar las variables independientes y la dependiente
Vars_Indep=df[["price", "accommodates", "bedrooms"]]
Var_Dep=df["host_response_rate"]
X=Vars_Indep
y=Var_Dep

#Dividimos el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=None)

#Se escalan los datos
escalar=StandardScaler()
X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

#Entrenar el modelo
algoritmo.fit(X_train, y_train)

y_pred=algoritmo.predict(X_test)
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print("Matriz de confusión")
print(matriz)

Matriz de confusión
[[2704    0]
 [  89    0]]


In [136]:
#calcular precision del modelo
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average="binary", pos_label="Respuesta alta")
print("precision del modelo:")
print(precision)

precision del modelo:
0.9681346222699606


In [137]:
#calcular exactitud
from sklearn.metrics import accuracy_score
exactitud=accuracy_score(y_test, y_pred)
print("exactitud del modelo")
print(exactitud)

exactitud del modelo
0.9681346222699606


In [139]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="Respuesta alta")
print("sensibilidad del modelo")
print(sensibilidad)

sensibilidad del modelo
1.0


host_total_listings_count

In [218]:
#Declarar las variables independientes y la dependiente
Vars_Indep=df[["price", "accommodates", "bedrooms"]]
Var_Dep=df["host_total_listings_count"]
X=Vars_Indep
y=Var_Dep

#Dividimos el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=None)

#Se escalan los datos
escalar=StandardScaler()
X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

#Entrenar el modelo
algoritmo.fit(X_train, y_train)

y_pred=algoritmo.predict(X_test)
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print("Matriz de confusión")
print(matriz)

Matriz de confusión
[[   0   11]
 [   0 2782]]


In [160]:
df2=df.groupby(["host_total_listings_count"])["host_total_listings_count"].count()
df2

<ipython-input-160-007228395fa2>:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df2=df.groupby(["host_total_listings_count"])["host_total_listings_count"].count()


,host_total_listings_count
host_total_listings_count,
Aceptacion baja,9275
Aceptacion alta,35


In [144]:
#calcular precision del modelo
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average="binary", pos_label="Aceptacion alta")
print("precision del modelo:")
print(precision)

precision del modelo:
0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [145]:
#calcular exactitud
from sklearn.metrics import accuracy_score
exactitud=accuracy_score(y_test, y_pred)
print("exactitud del modelo")
print(exactitud)

exactitud del modelo
0.9967776584317938


In [147]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="Aceptacion alta")
print("sensibilidad del modelo")
print(sensibilidad)

sensibilidad del modelo
0.0


number_of_reviews

In [148]:
#Declarar las variables independientes y la dependiente
Vars_Indep=df[["price", "accommodates", "bedrooms"]]
Var_Dep=df["number_of_reviews"]
X=Vars_Indep
y=Var_Dep

#Dividimos el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=None)

#Se escalan los datos
escalar=StandardScaler()
X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

#Entrenar el modelo
algoritmo.fit(X_train, y_train)

y_pred=algoritmo.predict(X_test)
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print("Matriz de confusión")
print(matriz)

Matriz de confusión
[[   0  124]
 [   0 2669]]


In [150]:
#calcular precision del modelo
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average="binary", pos_label="Reseñas altas")
print("precision del modelo:")
print(precision)

precision del modelo:
0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [151]:
#calcular exactitud
from sklearn.metrics import accuracy_score
exactitud=accuracy_score(y_test, y_pred)
print("exactitud del modelo")
print(exactitud)

exactitud del modelo
0.9556032939491587


In [153]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="Reseñas altas")
print("sensibilidad del modelo")
print(sensibilidad)

sensibilidad del modelo
0.0


latitude

In [154]:
#Declarar las variables independientes y la dependiente
Vars_Indep=df[["price", "accommodates", "bedrooms"]]
Var_Dep=df["latitude"]
X=Vars_Indep
y=Var_Dep

#Dividimos el conjunto de datos
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.30, random_state=None)

#Se escalan los datos
escalar=StandardScaler()
X_train=escalar.fit_transform(X_train)
X_test=escalar.fit_transform(X_test)

#definimos el algoritmo a utilizar
from sklearn.linear_model import LogisticRegression
algoritmo=LogisticRegression()

#Entrenar el modelo
algoritmo.fit(X_train, y_train)

y_pred=algoritmo.predict(X_test)
from sklearn.metrics import confusion_matrix
matriz=confusion_matrix(y_test, y_pred)
print("Matriz de confusión")
print(matriz)

Matriz de confusión
[[   0   94]
 [   0 2699]]


In [156]:
#calcular precision del modelo
from sklearn.metrics import precision_score
precision=precision_score(y_test, y_pred, average="binary", pos_label="Latitud altas")
print("precision del modelo:")
print(precision)

precision del modelo:
0.0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [157]:
#calcular exactitud
from sklearn.metrics import accuracy_score
exactitud=accuracy_score(y_test, y_pred)
print("exactitud del modelo")
print(exactitud)

exactitud del modelo
0.966344432509846


In [158]:
from sklearn.metrics import recall_score

sensibilidad=recall_score(y_test, y_pred, average="binary", pos_label="Latitud altas")
print("sensibilidad del modelo")
print(sensibilidad)

sensibilidad del modelo
0.0
